# Move tests

- Testing backend
- Grid scene 

In [1]:
from srcmodules import Objects, Robots, Tasks, Users
from srcmodules.Scenes import Scene
from srcmodules.Actions import Actions
from srcmodules.Gestures import Gestures
from srcmodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

Invalid MIT-MAGIC-COOKIE-1 key

In [2]:
s = Scene(init='cup')
s.info

Scene info. shape: [4, 4, 4]
0. cup,	cup,	[0.6 0.1 0.2], full,	|| [cup] >>
Robot: [0.6 0.1 0.2], opened, rotation: 1, attached: cup
User id: 0


### 1. Fake move

- `fake_handle_location` succeeds

In [3]:
Actions.do(s, ('pick_up','cup'), fake_handle_location=True)
s.info

Scene info. shape: [4, 4, 4]
0. cup,	cup,	[0.6 0.1 0.2], full,	|| [cup] >>
Robot: [0.6 0.1 0.2], opened, rotation: 1, attached: cup
User id: 0


- fails

In [4]:
s = Scene(init='cup,cup')
s.r.eef_position = s.cup.position
Actions.do(s, ('pick_up', 'cup'))

False

In [7]:
s.info

Scene info. shape: [4, 4, 4]
cup,	cup,	[1 2 0], full,	|| [cup] >>
cup1,	cup,	[2 0 1], full,	|| [cup1] >>
Robot: [1 2 0], opened, rotation: 1, attached: cup1


### 2. Move via actions
- Move actions can be executed independently `Actions.do(s, ('move_xxx', None))`, when set of actions contains the desired move actions

In [8]:
Actions.do(s, ('move_up',None))

True

In [9]:
s.info

Scene info. shape: [4, 4, 4]
cup,	cup,	[1 2 0], full,	|| [cup] >>
cup1,	cup,	[1 2 1], full,	|| [cup1] >>
Robot: [1 2 1], opened, rotation: 1, attached: cup1


### 3. Non-move actions by ignoring the location values

- `ignore_location` flag
- do not consider eef position

In [11]:
from srcmodules import Objects, Robots, Tasks, Users
from srcmodules.Scenes import Scene
from srcmodules.Actions import Actions
from srcmodules.Gestures import Gestures
from srcmodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

In [15]:
s = Scene(init='cup', random=False)
s.info

Scene info. shape: [4, 4, 4]
cup,	cup,	[0 0 0], empty,	|| [cup] >>
Robot: [0 0 3], opened, rotation: 0, attached: -


In [16]:
Actions.do(s, ('pick_up', 'cup'))

False

Robot eef and cup are not at one position, but action `pick_up` can be performed when `ignore_location=True`.

In [17]:
Actions.do(s, ('pick_up', 'cup'), ignore_location=True)

True

### 4. Non-move action by handeling the location

- `handle_location` flag
- Program will automatically execute move action sequence and move eef to desired object position
- Move actions are planned and executed from start (eef) to target (object)
- When there is obstacle on the scene, the some move actions are not valid for given eef position, the planner finds the way of valid actions to reach the target

In [19]:
s = Scene(init='cup', random=False)
Actions.do(s, ('pick_up', 'cup'), handle_location=True)

True

handle_location flag calls function `s.plan_path_to_position(..)` and then `Actions.execute_path_to_position(..)`
Example:

In [20]:
s = Scene(init='cup')
s.r.eef_position

array([1, 3, 3])

In [21]:
move_action_seq = s.plan_path_to_position([0,0,0], Actions)
Actions.execute_path_to_position(s,move_action_seq)

True

In [22]:
move_action_seq

['move_left',
 'move_down',
 'move_left',
 'move_down',
 'move_front',
 'move_left',
 'move_down']

In [23]:
s.r.eef_position

array([0, 0, 0])

In [24]:
Actions.get_possible_move_actions(s)

['move_back', 'move_right', 'move_up']

### Helper functions
- `get_possible_actions`
- `get_possible_actions_bool_array`

In [25]:
from srcmodules import Objects, Robots, Tasks, Users
from srcmodules.Scenes import Scene
from srcmodules.Actions import Actions
from srcmodules.Gestures import Gestures
from srcmodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

In [26]:
s = Scene(init='cup,cup,drawer,object', random=False)

In [27]:
Actions.get_possible_actions(s, ignore_location=True)

[('move_down', 'cup'),
 ('move_down', 'cup1'),
 ('move_down', 'drawer'),
 ('move_down', 'object'),
 ('move_right', 'cup'),
 ('move_right', 'cup1'),
 ('move_right', 'drawer'),
 ('move_right', 'object'),
 ('pick_up', 'cup'),
 ('pick_up', 'cup1'),
 ('pick_up', 'object'),
 ('open', 'drawer')]

In [28]:
Actions.get_possible_actions_bool_array(s, ignore_location=True)

array([[False, False, False, False],
       [False, False, False, False],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [ True,  True, False,  True],
       [False, False, False, False],
       [False, False,  True, False],
       [False, False, False, False]])

In [29]:
Actions.do(s,('pick_up', 'cup'), ignore_location=True)

True

In [30]:
Actions.get_possible_actions(s, ignore_location=True)

[('move_down', 'cup'),
 ('move_down', 'cup1'),
 ('move_down', 'drawer'),
 ('move_down', 'object'),
 ('move_right', 'cup'),
 ('move_right', 'cup1'),
 ('move_right', 'drawer'),
 ('move_right', 'object'),
 ('put', 'object'),
 ('put_on', 'object'),
 ('place', 'cup'),
 ('place', 'cup1'),
 ('place', 'drawer'),
 ('place', 'object'),
 ('open', 'drawer')]

In [31]:
Actions.get_possible_actions_bool_array(s, ignore_location=True)

array([[False, False, False, False],
       [False, False, False, False],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [False, False, False,  True],
       [False, False, False,  True],
       [False, False, False, False],
       [False, False, False, False],
       [ True,  True,  True,  True],
       [False, False,  True, False],
       [False, False, False, False]])